# Synthetic basaltic glasses

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Set up to find custom python package
import os
import sys
import numpy as np
sys.path.insert(1, ".")
sys.path.insert(1, "..")

In [ ]:
from src import readfiles, wdscan, correct_quant, calczaf, helper_funs
import pickle
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

In [ ]:
# For reference: how to rename folders

# from src.readfiles import rename_folders_as_comments

# for f in os.listdir("../data/raw/basaltic_glasses_StA/raw_quant_20211222/original/"):
#     print(f)
                    
#     rename_folders_as_comments(
#         Path(f"../data/raw/basaltic_glasses_StA/raw_quant_20211222/original/{f}"), 
#         "../data/raw/basaltic_glasses_StA/raw_quant_20211222/renamed")

In [ ]:
import pyrolite.geochem

# Major/trace elements other than N

In [ ]:
def get_major_elements_from_txt(path):

    majors_raw = pd.read_csv(
        path,
        header=1,
        index_col=False
        )[:5]

    mass_pct_cols = [col for col in majors_raw.columns if "(Mass%)" in col]
    majors = majors_raw.loc[:, mass_pct_cols].rename(columns={col: col.replace("(Mass%)", "") for col in mass_pct_cols})
    detlims = majors_raw[[col for col in majors_raw.columns if "(D.L.)" in col]]
    return majors, detlims

majors = {}
detlims = {}
for s in ["Edi06", "Edi09", "D2872"]:

    majors[s], detlims[s] = get_major_elements_from_txt(
        f"../data/raw/basaltic_glasses_StA/majors_20211110/{s.lower()}_all.txt"
    )

# for D2983 I didn't get an "_all.txt" file so I need to get the data differently

majors["D2893"] = pd.read_csv(
    "../data/raw/basaltic_glasses_StA/majors_20211110/D2893_2_oxide.csv",
    header=1,
    index_col=False
)[:4]

majors["D2893"].columns = [c.strip() for c in majors["D2893"].columns]

detlims["D2893"] = pd.read_csv(
    "../data/raw/basaltic_glasses_StA/majors_20211110/D2893_2_detlim.txt",
    header=1,
    index_col=False
)[:4]

In [ ]:
majors["D2872"].columns

In [ ]:
majors["D2893"].columns

In [ ]:
majors_element = {}

for k, df in majors.items():

    majors_element[k] = df[
        ['SiO2', 'Al2O3', 'Cl', 'P2O5', 'Fe2O3', 'MnO', 'Cr2O3', 'K2O', 'CaO', 'Ru2O3', 'Na2O', 'MgO']
        ].pyrochem.convert_chemistry(
        to=['Si', 'Al', 'Cl', 'P', 'Fe', 'Mn', 'Cr', 'K', 'Ca', 'Ru', 'Na', 'Mg']
    ).fillna(0)

    majors_element[k]["O"] = df["Total"] - (majors_element[k].sum(axis=1))

    majors_element[k]["Total"] = majors_element[k].sum(axis=1)


In [ ]:
majors_element

In [ ]:
majors_summary = {}

for k, df in majors_element.items():
    majors_summary[k] = pd.concat([df.T.mean(axis=1), df.T.std(axis=1)], axis=1)
    majors_summary[k].columns = ["wt% mean", "stdev"]

majors_summary_combined = pd.concat(majors_summary, axis=1)

In [ ]:
majors_summary_combined.round(3).to_csv("../data/processed/basaltic_glasses/basaltic_glasses_majors_summary.csv")
majors_summary_combined.round(3)

In [ ]:
detlims["Edi09"]

# Nitrogen analyses

### WD scan - visualise & fit

D2872 - all dates

In [ ]:
samplenames = ["D2872", "D2893", "Edi09", "Edi06"]
metadata_list = {}
data_list = {}

for s in samplenames:
    print(f"---------------- {s} ----------------")

    folderpath_list = [
        f"../data/raw/basaltic_glasses_StA/wd_scans_20211008/{s}",
        f"../data/raw/basaltic_glasses_StA/wd_scans_20211125/{s}",
        f"../data/raw/basaltic_glasses_StA/wd_scans_20211222/{s}"
    ]

    metadata_list[s] = []
    data_list[s] = []

    for i, f in enumerate(folderpath_list):
        # Read in the data
        for d in ["data001", "data002"]:
            try:             
                comments, data, metadata = readfiles.import_jeol_wdscans(
                    subfolder=f,
                    scan_filename=f'{d}_mm.csv',
                    cnd_filename=f'{d}.cnd',
                    comment_line_num=80,
                    crystal_line_name="$XM_WDS_CRYSTAL_NAME%0",
                    sep=',',
                    return_metadata=True
                )

                metadata_list[s].append(metadata)
                data_list[s].append(data)
            except FileNotFoundError:
                 print(f"No file found for {d}")


In [ ]:
lde1l_scans = {}
lde1_scans = {}

for s in samplenames:
    print(f"--------------------{s}-------------------")
    lde1l_scans[s] = {"metadata": [], "data": []}
    lde1_scans[s] = {"metadata": [], "data": []}

    for i in range(len(data_list[s])):
        if metadata_list[s][i].crystal == "LDE1L":
            lde1l_scans[s]["metadata"].append(metadata_list[s][i])
            lde1l_scans[s]["data"].append(data_list[s][i])

        if metadata_list[s][i].crystal == "LDE1":
            lde1_scans[s]["metadata"].append(metadata_list[s][i])
            lde1_scans[s]["data"].append(data_list[s][i])    


    fig, ax = plt.subplots(ncols=4, figsize=(10, 3))

    for i in range(len(lde1l_scans[s]["data"])):
            data = lde1l_scans[s]["data"][i]

            ax[i].plot(data.L, data.cps_per_nA, ".k", markersize=1)
            ax[i].set_title(s)

            ax[3].plot(data.L, data.cps_per_nA, ".", markersize=1, label=i)
            ax[3].set_title("All scans overlain")

    display(pd.DataFrame(lde1l_scans[s]["metadata"]).T)
    plt.show()

For each of these, we can just fit the middle scan - the highest-resolution one
Except for Edi06 for which there is only a single scan

In [ ]:
# Fit and plot with the fits ------------
# Choose parts of the spectrum to use in the fit
bg_roi = [[120,138], [155, 180]]
sample = "D2872"

comments, data, metadata = readfiles.import_jeol_wdscans(
    subfolder="../data/raw/basaltic_glasses_StA/wd_scans_20211125/D2872",
    scan_filename='data001_mm.csv',
    cnd_filename='data001.cnd',
    comment_line_num=80,
    crystal_line_name="$XM_WDS_CRYSTAL_NAME%0",
    sep=',',
    return_metadata=True
)
print(metadata.crystal)
trimmed_data = wdscan.trim_data_from_regions(data, bg_roi)
out = wdscan.fit_bg(trimmed_data)
wdscan.plot_bg_fit(data, trimmed_data, out, sample, [146.6], save_to=Path("../data/interim/basaltic_glasses/fits"))
par_dict = wdscan.write_fit_params(out, sample, save_to=Path("../data/interim/basaltic_glasses/fits/"))

## D2983

In [ ]:
# Fit and plot with the fits ------------
# Choose parts of the spectrum to use in the fit
bg_roi = [[120,140], [170, 180]]
sample = "D2893"

comments, data, metadata = readfiles.import_jeol_wdscans(
    subfolder="../data/raw/basaltic_glasses_StA/wd_scans_20211125/D2893",
    scan_filename='data001_mm.csv',
    cnd_filename='data001.cnd',
    comment_line_num=80,
    crystal_line_name="$XM_WDS_CRYSTAL_NAME%0",
    sep=',',
    return_metadata=True
)

print(metadata.crystal)
trimmed_data = wdscan.trim_data_from_regions(data, bg_roi)
out = wdscan.fit_bg(trimmed_data)
wdscan.plot_bg_fit(data, trimmed_data, out, sample, [146.6], save_to=Path("../data/interim/basaltic_glasses/fits"))
par_dict = wdscan.write_fit_params(out, sample, save_to=Path("../data/interim/basaltic_glasses/fits/"))

## Edi09

In [ ]:
# Fit and plot with the fits ------------
# Choose parts of the spectrum to use in the fit
bg_roi = [[120,140], [170, 180]]
sample = "Edi09"

comments, data, metadata = readfiles.import_jeol_wdscans(
    subfolder="../data/raw/basaltic_glasses_StA/wd_scans_20211125/Edi09",
    scan_filename='data001_mm.csv',
    cnd_filename='data001.cnd',
    comment_line_num=80,
    crystal_line_name="$XM_WDS_CRYSTAL_NAME%0",
    sep=',',
    return_metadata=True
)

print(metadata.crystal)
trimmed_data = wdscan.trim_data_from_regions(data, bg_roi)
out = wdscan.fit_bg(trimmed_data)
wdscan.plot_bg_fit(data, trimmed_data, out, sample, [146.6], save_to=Path("../data/interim/basaltic_glasses/fits"))
par_dict = wdscan.write_fit_params(out, sample, save_to=Path("../data/interim/basaltic_glasses/fits/"))

## Edi06

In [ ]:
# Fit and plot with the fits ------------
# Choose parts of the spectrum to use in the fit
bg_roi = [[120,140], [170, 180]]
sample = "Edi06"

comments, data, metadata = readfiles.import_jeol_wdscans(
    subfolder="../data/raw/basaltic_glasses_StA/wd_scans_20211222/Edi06",
    scan_filename='data001_mm.csv',
    cnd_filename='data001.cnd',
    comment_line_num=80,
    crystal_line_name="$XM_WDS_CRYSTAL_NAME%0",
    sep=',',
    return_metadata=True
)

print(metadata.crystal)
trimmed_data = wdscan.trim_data_from_regions(data, bg_roi)
out = wdscan.fit_bg(trimmed_data)
wdscan.plot_bg_fit(data, trimmed_data, out, sample, [146.6], save_to=Path("../data/interim/basaltic_glasses/fits"))
par_dict = wdscan.write_fit_params(out, sample, save_to=Path("../data/interim/basaltic_glasses/fits/"))